In [1]:
import timeit

import pandas as pd
import mysql.connector

# Carregamento

In [2]:
df_viagem = pd.read_csv('./dataset/gerados/viagem.csv')
df_passagem = pd.read_csv('./dataset/gerados/passagem.csv')
df_pagamento = pd.read_csv('./dataset/gerados/pagamento.csv')
df_cpf = pd.read_csv('./dataset/gerados/cpf.csv')
df_puc = pd.read_csv('./dataset/gerados/puc.csv')
df_paises = pd.read_csv('./dataset/gerados/paises.csv')
df_uf = pd.read_csv('./dataset/gerados/uf.csv')
df_cidades = pd.read_csv('./dataset/gerados/cidades.csv')
df_pessoa = pd.read_csv('./dataset/gerados/pessoa.csv')
df_cargo = pd.read_csv('./dataset/gerados/cargo.csv')
df_funcao = pd.read_csv('./dataset/gerados/funcao.csv')
df_ministerios = pd.read_csv('./dataset/gerados/ministerio.csv')
df_meio = pd.read_csv('./dataset/gerados/meio.csv')

# ETL

In [3]:
cnx = mysql.connector.connect(user='root', password='root',
                              host='localhost',
                              database='VIAGENS_DB')

In [4]:
cursor = cnx.cursor()

In [5]:
# Para 2 campos: ID(padrao) e NOME
dict_colunas = {'PAIS': df_paises,
                'CIDADE': df_cidades, 
                'UF': df_uf, 
                'MEIO_TRANSPORTE': df_meio, 
                'CARGO': df_cargo,
               }

In [6]:
%%time
for name, df in dict_colunas.items():
    print(name)
    for _, row in enumerate(df.itertuples()):
        query = f"INSERT INTO {name} "\
                 f"(NOME)" \
                 f"VALUES ({row.nome!r})"
        cursor.execute(query)

PAIS
CIDADE
UF
MEIO_TRANSPORTE
CARGO
CPU times: user 44.9 ms, sys: 19.4 ms, total: 64.3 ms
Wall time: 172 ms


In [7]:
cnx.commit()

In [8]:
%%time
# Ministerio
for _, row in enumerate(df_ministerios.itertuples()):
    query = f"INSERT INTO MINISTERIO"\
             f"(ID_MINISTERIO, NOME)" \
             f"VALUES ({row.id_ministerio!r}, {row.nome!r})"
    cursor.execute(query)

CPU times: user 2.4 ms, sys: 254 µs, total: 2.65 ms
Wall time: 3.7 ms


In [9]:
cnx.commit()

In [10]:
%%time
# Pessoa
for _, row in enumerate(df_pessoa.itertuples()):
    query = f"INSERT INTO PESSOA"\
             f"(NOME, CPF)" \
             f"VALUES ({row.nome!r}, {row.cpf!r})"
    cursor.execute(query)

CPU times: user 2.81 s, sys: 1.2 s, total: 4.01 s
Wall time: 11.5 s


In [11]:
cnx.commit()

In [12]:
%%time
# Funcao
for _, row in enumerate(df_funcao.itertuples()):
    query = f"INSERT INTO FUNCAO"\
             f"(NOME, DESCRICAO)" \
             f"VALUES ({row.nome!r}, {row.descricao!r})"
    cursor.execute(query)

CPU times: user 2.17 ms, sys: 3.49 ms, total: 5.66 ms
Wall time: 13.5 ms


In [13]:
cnx.commit()

In [14]:
%%time
# CPF
for _, row in enumerate(df_cpf.itertuples()):
    query = f"INSERT INTO PESSOA_CARGO_FUNCAO"\
             f"(ID_PESSOA, ID_FUNCAO, ID_CARGO)" \
             f"VALUES ({row.id_pessoa!r}, {row.id_funcao!r}, {row.id_cargo!r})"
    cursor.execute(query)

CPU times: user 3.08 s, sys: 1.36 s, total: 4.44 s
Wall time: 12.9 s


In [15]:
cnx.commit()

In [16]:
%%time
# PUC
for _, row in enumerate(df_puc.itertuples()):
    query = f"INSERT INTO PAIS_CIDADE_UF"\
             f"(ID_PAIS, ID_UF, ID_CIDADE)" \
             f"VALUES ({row.id_pais!r}, {row.id_uf!r}, {row.id_cidade!r})"
    cursor.execute(query)

CPU times: user 15.5 ms, sys: 10.1 ms, total: 25.6 ms
Wall time: 82.7 ms


In [17]:
cnx.commit()

In [18]:
%%time
# VIAGEM
for _, row in enumerate(df_viagem.itertuples()):
    query = f"INSERT INTO VIAGEM"\
             f"(ID_VIAGEM, IS_REALIZADO, IS_URGENTE, DATA_INICIO, DATA_FIM, ID_MINISTERIO, ID_PESSOA_CARGO_FUNCAO, TOTAL)" \
             f"VALUES ({row.id_viagem!r}, {row.is_realizado!r}, {row.is_urgente!r}, {row.data_inicio!r}, {row.data_fim!r}, {row.id_ministerio!r}, {row.id_cpf!r}, {row.total!r})"
    cursor.execute(query)

CPU times: user 8.74 s, sys: 3.37 s, total: 12.1 s
Wall time: 47.6 s


In [19]:
cnx.commit()

In [20]:
%%time
# PASSAGEM
for _, row in enumerate(df_passagem.itertuples()):
    query = f"INSERT INTO PASSAGEM"\
             f"(ID_VIAGEM, ID_MEIO_TRANSPORTE, ID_PAIS_CIDADE_UF_ORIGEM_IDA, ID_PAIS_CIDADE_UF_DESTINO_IDA, ID_PAIS_CIDADE_UF_ORIGEM_VOLTA, ID_PAIS_CIDADE_UF_DESTINO_VOLTA)" \
             f"VALUES ({row.id_viagem!r}, {row.id_meio!r}, {row.id_puc_origem_ida!r}, {row.id_puc_destino_ida!r}, {row.id_puc_origem_volta!r}, {row.id_puc_destino_volta!r})"
    cursor.execute(query)

CPU times: user 2.98 s, sys: 1.22 s, total: 4.2 s
Wall time: 13.4 s


In [21]:
cnx.commit()

In [22]:
%%time
# Pagamento
for _, row in enumerate(df_pagamento.itertuples()):
    print(_, end='\r')
    query = f"INSERT INTO PAGAMENTO"\
             f"(VALOR, TAXA, ID_PASSAGEM)" \
             f"VALUES ({row.valor!r}, {row.taxa!r}, {row.id_passagem!r})"
    cursor.execute(query)

CPU times: user 11.1 s, sys: 2.84 s, total: 14 s
Wall time: 15.8 s


In [23]:
cnx.commit()

In [ ]:
cursor.reset()

In [24]:
cnx.close()